In [ ]:
!ls

blues.00001.wav      inspirational-symphony-classical-music-loop-243157.mp3  sample_data
classical.00000.wav  MusicGenre_CNN_79.73.h5


In [ ]:
import librosa
import numpy as np
from tensorflow import keras
import librosa
import numpy as np
from tensorflow import keras
import math # Import the math module

# ... (rest of your code) ...
# Load the saved model
model = keras.models.load_model("MusicGenre_CNN_79.73.h5")

# Function to preprocess the audio file
def process_input(audio_file, track_duration):
    SAMPLE_RATE = 22050
    NUM_MFCC = 13
    N_FTT = 2048
    HOP_LENGTH = 512
    TRACK_DURATION = track_duration  # measured in seconds
    SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION
    NUM_SEGMENTS = 10

    samples_per_segment = int(SAMPLES_PER_TRACK / NUM_SEGMENTS)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / HOP_LENGTH)

    signal, sample_rate = librosa.load(audio_file, sr=SAMPLE_RATE)

    # Extract MFCCs for the first segment
    start = samples_per_segment * 0
    finish = start + samples_per_segment
    mfcc = librosa.feature.mfcc(y=signal[start:finish], sr=sample_rate, n_mfcc=NUM_MFCC, n_fft=N_FTT, hop_length=HOP_LENGTH)

    mfcc = mfcc.T

    return mfcc

# Dictionary to map predicted index to genre
genre_dict = {
    0: "disco",
    1: "pop",
    2: "classical",
    3: "metal",
    4: "rock",
    5: "blues",
    6: "hiphop",
    7: "reggae",
    8: "country",
    9: "jazz",
}

# Path to your audio file
audio_file_path = "inspirational-symphony-classical-music-loop-243157.mp3"

# Preprocess the audio
new_input_mfcc = process_input(audio_file_path, 30)

# Reshape the input for the model
X_to_predict = new_input_mfcc[np.newaxis, ..., np.newaxis]

# Make the prediction
prediction = model.predict(X_to_predict)

# Get the predicted genre
predicted_index = np.argmax(prediction, axis=1)
predicted_genre = genre_dict[int(predicted_index)]

print("Predicted Genre:", predicted_genre)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
Predicted Genre: reggae


<ipython-input-13-891d308d3ec8>:65: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_genre = genre_dict[int(predicted_index)]


In [3]:
!pip install gradio
import gradio as gr  # Import Gradio for building the UI
import librosa  # For audio processing
import numpy as np  # For numerical operations
from tensorflow import keras  # For loading the trained model
import math  # For mathematical calculations

# Load the pre-trained model for genre classification
model = keras.models.load_model("MusicGenre_CNN_79.73.h5")

# Function to preprocess the audio file
def process_input(audio_file, track_duration=30):
    SAMPLE_RATE = 22050  # Sample rate for audio processing
    NUM_MFCC = 13  # Number of MFCCs to extract
    N_FTT = 2048  # Number of samples per FFT
    HOP_LENGTH = 512  # Number of samples between successive frames
    TRACK_DURATION = track_duration  # Duration of the audio to process (in seconds)
    SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION  # Total samples in the track
    NUM_SEGMENTS = 10  # Number of segments to split the audio into

    # Calculate samples and MFCC vectors per segment
    samples_per_segment = int(SAMPLES_PER_TRACK / NUM_SEGMENTS)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / HOP_LENGTH)

    # Load the audio file and resample it
    signal, sample_rate = librosa.load(audio_file, sr=SAMPLE_RATE)

    # Extract MFCCs for the first segment
    start = samples_per_segment * 0  # Start index of the first segment
    finish = start + samples_per_segment  # End index of the first segment
    mfcc = librosa.feature.mfcc(
        y=signal[start:finish],
        sr=sample_rate,
        n_mfcc=NUM_MFCC,
        n_fft=N_FTT,
        hop_length=HOP_LENGTH,
    )

    # Transpose the MFCC array to make it suitable for the model
    mfcc = mfcc.T
    return mfcc

# Dictionary to map the model's predicted index to a genre
genre_dict = {
    0: "disco",
    1: "pop",
    2: "classical",
    3: "metal",
    4: "rock",
    5: "blues",
    6: "hiphop",
    7: "reggae",
    8: "country",
    9: "jazz",
}

# Function to predict the genre of the audio file
def predict_genre(audio_file):
    # Preprocess the uploaded audio file
    new_input_mfcc = process_input(audio_file, 30)
    # Reshape the input to match the model's expected input shape
    X_to_predict = new_input_mfcc[np.newaxis, ..., np.newaxis]
    # Make the prediction using the loaded model
    prediction = model.predict(X_to_predict)
    # Get the genre index with the highest probability
    predicted_index = np.argmax(prediction, axis=1)
    # Map the predicted index to the genre name
    predicted_genre = genre_dict[int(predicted_index)]
    return f"Predicted Genre: {predicted_genre}"

# Create the Gradio interface
interface = gr.Interface(
    fn=predict_genre,  # Function to be executed when user uploads a file
    inputs=gr.Audio( type="filepath"),  # Accept uploaded audio files
    outputs="text",  # Display the prediction as text
    title="Music Genre Classifier",  # Title of the app
    description="Upload an audio file to predict its genre.",  # Short description
)

# Launch the Gradio app
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://04318c89952905a27d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
